In [138]:
import pandas as pd

users = pd.read_csv("../data/Users.csv")

users

,documentid,paymentid,amount,date,vendor_name,vendor_address
0,00d0624439175,00p0535380313,72.90,2916-4-17,MR. D.I.Y. (KUCHZAI) SDNBHD,"LOT 1851-A & 1851-B, JALAN KPB 6, KAWASAN PERI..."
1,00d0654270874,00p0432205035,43.90,2016-6-7,FOUR QUARTERS SDN BHD,NO.28G JLN SS21/35 DAMANSARA UTAMA PETALING JA...
2,00d0403711886,00p0174820163,32.23,2018-10-1,SYARIKAT PERNIAGAAN GIJ KEE,"NO 290, JALAN AIR PANAS, SETAPAK, 53020, KUALA..."
3,00d0956164753,00p0943784617,503.10,2018-3-23,TIMELESS KITCHENETE SDN BHD,"LLT 50, FLOOR T2,SKY AVENUE GENTING HIGHLANDS ..."
4,00d0667839698,00p0762437501,1110.00,2016-2-28,SIN THEY & COMPANY,"NO.54, BATU 5,JALAN IPOH, KG.BATU, 5110 KUALA ..."
...,...,...,...,...,...,...
494,00d0223498663,00p0611347643,29.30,2016-12-25,GOLDEN ARCHES RESTAURANTS SDN BHD,"LEVEL 6, BANGUNAN TH, DAMANSARA UPTOWN3 NO.3, ..."
495,00d0206227401,00p0603265655,8.70,2017-8-7,SANYU STATIONERY SHOP,"NO. 31G&3G, JALAN SETIA INDAH X ,U13/X 40170 S..."
496,00d0898930568,00p0751192439,2.50,2018-1-24,99 SPEED MART S/B,"LOT P.T. 281 1, JALAN ANGSA, TAMAN BERKELEY 41..."
497,00d0397644256,00p0962484273,63.80,2018-2-22,BILLION SIX ENTERPRISE,"NO 3, JALAN TAMAN JAS'A 2; . SECTION U6, 40150..."


In [139]:
from fuzzywuzzy import fuzz
from dateutil import parser


numerics = set(['0', '1', '2', '3', '4', '5', '6', '7', '8', '9', '.'])

def get_score(features, transaction):
    score = 0

    i = 0
    for ex_ft, tr_ft in zip(features, transaction):
        if pd.isna(ex_ft) or i < 2:
            i += 1
            continue

        ex_ft = ex_ft.lower().strip()
        tr_ft = str(tr_ft).lower().strip()
        
        if i == 2:  # amount
            ex_ft = ''.join([c for c in ex_ft if c in numerics])

        if i == 3:  # date
            ex_ft = ex_ft.split()[0]
            date = parser.parse(ex_ft)
            ex_ft = date.strftime("%Y-%-m-%-d")
            # print(date)

        ratio = fuzz.token_sort_ratio(ex_ft, tr_ft) / 100.0

        if i == 2:  # amount
            if ratio >= 0.5:
                score += 10 * ratio
            else:
                score += 2 * ratio

        if i == 3:  # date
            if ratio > 0.7:
                score += 10 * ratio
            elif ratio > 0.5:
                score += 5 * ratio
        
        if i == 4:  # vendor name
            if ratio > 0.8:
                score += 20 * ratio
            elif ratio > 0.5:
                score += 5 * ratio

        if i == 5:  # vendor address
            if ratio > 0.8:
                score += 20 * ratio
            elif ratio > 0.5:
                score += 5 * ratio

        # print(ratio, ex_ft, tr_ft)

        i += 1
    return score
    # print(score)

In [140]:
df = pd.read_pickle("./dataframe.pkl")

columns_titles = ["doc_id", "payment_id", "S-TOTAL", "S-DATE", "S-COMPANY", "S-ADDRESS"]
df=df.reindex(columns=columns_titles)

df = df.rename(columns={"doc_id": "documentid", "payment_id": "paymentid", "S-TOTAL": "amount", "S-DATE": "date", "S-COMPANY": "vendor_name", "S-ADDRESS": "vendor_address"})

df

,documentid,paymentid,amount,date,vendor_name,vendor_address
0,00d0377951457,00p0693154757,7.60,<NA>,TRIPLE SIX POINT ENTERPRISE 666,NO 14& 16 JALAN PERMAS 4/3 BANDAR BARU PERMAS...
1,00d0660852423,00p0965564934,10.30,<NA>,LEMON TREE RESTAURANT,NO 3 BANDAR BARU PERMAS JAYA
2,00d0803190070,00p0247334622,<NA>,<NA>,SANYU STATIONERY SHOP,NO. 31G&33G 40170 SETIA ALAM
3,00d0259552507,00p0876493998,<NA>,05-JAN-2017 03:17:50 PM,S&Y STATIONERY,BANDAR PINGGIRAN SUBANG SEKSYEN U5 40150 SHAH...
4,00d0886341707,00p0151413045,<NA>,<NA>,DIMILIKI OLEH : DOVE HOLDINGS SDN BHD,LOT S25 NO 1 81100 JOHOR BAHRU
...,...,...,...,...,...,...
557,00d0880901672,00p0741339461,<NA>,19 APR 2018 18:22,UNIHAKKA INTERNATIONAL SDN BHD,<NA>
558,00d0677299556,00p0504275506,<NA>,11-05-16,99 SPEED MART S/B (519537-X),LOT P.T. 33198 JALAN KAPAR 42100 KLANG 1605-T...
559,00d0242366463,00p0778749877,RM7.90 RM8.35,<NA>,YAM FRESH,NO.145G LAMAN RIMBUNAN KEPONG 52100 KUALA LUMPUR
560,00d0504003816,00p0361841562,4.60,<NA>,RESTORAN WAN SHENG,NO.2 SEKSYEN 9 43200 CHERAS


In [141]:
test_users = pd.read_csv("../data/test_transactions.csv")

test_users

,documentid,paymentid,amount,date,vendor_name,vendor_address
0,00d0472780579,00p0878907338,7.80,2018-4-4,RESTTORAN WAN SHENG,"NO.2, JALAN TEMENGUNG 19/9, SEKSYEM 9, BANDAR ..."
1,00d0269258457,00p0953953549,24.10,2017-9-13,AIK HUAT HARDWARE ENTERRPISE (SETIA ALAM) SDN BHD,"NO. 17-G, JALAN SETIA INDAH (X) U13/X, SETIA A..."
2,00d0145054354,00p0128923472,7.00,2017-11-12,SANYU STATIONERY SHOP,"NO. 31G&33G, JALAN SEITA INDAH X ,U13/X 40170 ..."
3,00d0795468800,00p0105962640,2.50,2018-3-5,SNG KEE CHSERAS RESTAURANT,"69, JALANTEMENGGUNG 3/9 BANDAR MAHKOTA CHERAS ..."
4,00d0682248746,00p0398282542,27.90,2018-1-28,99 SPEED MART S/B,"LOT P.T. 2811, JALAN ANGSA, TAAMN BERKELEY 411..."
...,...,...,...,...,...,...
121,00d0413686396,00p0979468067,18.90,2018-11-5,GERBANG ALAF RESTAURAMTS SDN BHD,"LEVEL 6, BANGUNAAN TH, DAMANSARA UPTOWN3 NO.3,..."
122,00d0745331693,00p0257799448,7.45,2018-4-26,KEDAI PAPAN YEW CHUAN,"LOT 276 JALAN BANTING 43800 DENGKIL, SELAMGOR"
123,00d0978814485,00p0847177660,227.90,2018-11-5,RESTAURANT JIAWEI JIAWEI HOUSE,"13, JLN TASIK UTAMA 8 MEDAN NIAGA DAMAI SG BES..."
124,00d0902251681,00p0637100547,7.50,2018-6-27,SANJUNG RALITI SDM. BHD.,"CAR PARK OFFICE, BASEMENT 1, LABUHRAYA SPRINT,..."


In [151]:
import heapq

n_correct = 0
top_k_correct = 0
total = 0
K = 10

for i, tr_row in test_users.iterrows():
    max_score = -1.0
    max_row = None
    heap = []
    for j, ft_row in df.iterrows():
        score = get_score(ft_row, tr_row)
        if score > max_score:
            max_row = ft_row
            max_score = score

        if ft_row.at['documentid'] == tr_row.at['documentid']:
            heapq.heappush(heap, (-score, True, ft_row.at['documentid']))
        else:
            heapq.heappush(heap, (-score, False, ft_row.at['documentid']))

    if max_row.at['documentid'] == tr_row.at['documentid']:
        n_correct += 1

    top_k = heapq.nlargest(K, heap, key=lambda x: -x[0])
    # print(top_k[0][2])
    # print(max_row[0])

    top_k_bool = list(map(lambda x: x[1], top_k))

    if (any(top_k_bool)):
        top_k_correct += 1

    # print(top_k_bool)

    total += 1

    # print(max_score)
    # print(max_row)
    # print(tr_row)
    # print()
    # break


print("Accuracy @ 1:", n_correct / total)
print(f"Accuracy @ {K}:", top_k_correct / total)


Accuracy @ 1: 0.5952380952380952
Accuracy @ 10: 0.8015873015873016
